In [1]:
import sys
sys.path.append('..')

from tqdm import tqdm

from processing.images_scrapper import ImageScrapper

In [2]:
QUERIES = {
    "apple": ["яблоко", "спелое яблоко", "зеленое яблоко", "спелое зеленое яблоко", "красное яблоко", "спелое красное яблоко", "яблоко на дереве", "сочное яблоко", "яблоки в корзинке", "красивые яблоки", "фото яблока", "целое яблоко", "apple", "ripe apple", "green apple", "ripe green apple", "red apple", "ripe red apple", "apple on tree", "juicy apple", "apples in basket", "beautiful apples", "apple photo", "whole apple", "fresh apple", "organic apple", "sweet apple", "natural apple", "garden apple", "autumn apple", "summer apple", "spring apple", "яблоко в саду", "садовое яблоко", "натуральное яблоко", "свежее яблоко", "сладкое яблоко"],
    "orange": ["апельсин", "спелый апельсин", "большой апельсин", "апельсин на дереве", "апельсины", "фото апельсина", "апельсины в корзинке", "целый апельсин", "orange", "ripe orange", "big orange", "orange on tree", "oranges", "orange photo", "oranges in basket", "whole orange", "juicy orange", "fresh orange", "sweet orange", "citrus orange", "natural orange", "organic orange", "сочный апельсин", "свежий апельсин", "сладкий апельсин", "цитрусовый апельсин", "натуральный апельсин", "органический апельсин", "апельсиновый сад", "orange grove", "valencia orange", "navel orange"],
    "banana": ["banana", "yellow bananas", "green bananas", "bunch of bananas", "bananas in skin", "unpeeled bananas", "ripe banana", "big banana", "banana on tree", "bananas", "banana photo", "bananas in basket", "whole banana", "fresh banana", "organic banana", "natural banana", "sweet banana", "tropical banana", "banana plant", "banana fruit", "банан", "желтый банан", "зеленый банан", "связка бананов", "спелый банан", "свежий банан", "тропический банан", "бананы в кожуре", "банановая гроздь"],
    "pear": ["груша", "зеленая груша", "желтая груша", "красивые груши", "спелая груша", "груша на дереве", "груши", "фото груши", "груши в корзинке", "целая груша", "pear", "green pear", "yellow pear", "beautiful pears", "ripe pear", "pear on tree", "pears", "pear photo", "pears in basket", "whole pear", "fresh pear", "sweet pear", "organic pear", "natural pear", "autumn pear", "bartlett pear", "conference pear", "anjou pear", "сочная груша", "свежая груша", "сладкая груша", "осенняя груша", "грушевый сад", "pear orchard"],
    "lemon": ["лемон", "желтый лемон", "зеленый лемон", "большой красивый лемон", "лемоны в корзинке", "целый лемон", "много лемонов", "фото лемона", "лемоны на дереве", "lemon", "yellow lemon", "green lemon", "big beautiful lemon", "lemons in basket", "whole lemon", "many lemons", "lemon photo", "lemons on tree", "fresh lemon", "citrus lemon", "organic lemon", "natural lemon", "meyer lemon", "свежий лемон", "цитрусовый лемон", "органический лемон", "лимонное дерево", "lemon tree", "sicilian lemon", "лимонный сад", "lemon grove"],
    "grape": ["виноград", "спелый виноград", "зеленый виноград", "красный виноград", "гроздь винограда", "виноград на лозе", "виноград в корзине", "фото винограда", "grape", "ripe grapes", "green grapes", "red grapes", "bunch of grapes", "grapes on vine", "grapes in basket", "grape photo", "purple grapes", "wine grapes", "table grapes", "fresh grapes", "organic grapes", "vineyard grapes", "фиолетовый виноград", "винный виноград", "столовый виноград", "свежий виноград", "органический виноград", "виноградник", "виноградная лоза", "grape vine"],
    "watermelon": ["арбуз", "спелый арбуз", "большой арбуз", "сочный арбуз", "целый арбуз", "арбуз на бахче", "разрезанный арбуз", "фото арбуза", "watermelon", "ripe watermelon", "big watermelon", "juicy watermelon", "whole watermelon", "watermelon field", "cut watermelon", "watermelon photo", "fresh watermelon", "sweet watermelon", "organic watermelon", "natural watermelon", "summer watermelon", "seedless watermelon", "свежий арбуз", "сладкий арбуз", "органический арбуз", "летний арбуз", "бессемянный арбуз", "арбузная плантация", "watermelon plantation"],
    "pineapple": ["ананас", "спелый ананас", "желтый ананас", "большой ананас", "целый ананас", "ананас на растении", "фото ананаса", "ананасы", "pineapple", "ripe pineapple", "yellow pineapple", "big pineapple", "whole pineapple", "pineapple plant", "pineapple photo", "pineapples", "fresh pineapple", "sweet pineapple", "tropical pineapple", "organic pineapple", "natural pineapple", "golden pineapple", "свежий ананас", "сладкий ананас", "тропический ананас", "органический ананас", "золотой ананас", "ананасовая плантация", "pineapple plantation"],
    "kiwi": ["киви", "спелый киви", "зеленый киви", "разрезанный киви", "целый киви", "киви на ветке", "фото киви", "много киви", "kiwi", "ripe kiwi", "green kiwi", "cut kiwi", "whole kiwi", "kiwi on branch", "kiwi photo", "many kiwis", "fresh kiwi", "sweet kiwi", "organic kiwi", "natural kiwi", "kiwi fruit", "golden kiwi", "свежий киви", "сладкий киви", "органический киви", "золотой киви", "киви фрукт", "плантация киви", "kiwi plantation", "kiwi vine"],
    "tomato": ["помидор", "спелый помидор", "красный помидор", "зеленый помидор", "помидоры на кусте", "помидоры в корзине", "фото помидора", "целый помидор", "tomato", "ripe tomato", "red tomato", "green tomato", "tomatoes on plant", "tomatoes in basket", "tomato photo", "whole tomato", "fresh tomato", "organic tomato", "cherry tomato", "plum tomato", "beef tomato", "vine tomato", "свежий помидор", "органический помидор", "черри помидор", "сливовидный помидор", "бычье сердце", "помидоры на ветке", "томатный куст", "tomato plant"],
    "cucumber": ["огурец", "зеленый огурец", "спелый огурец", "огурцы на грядке", "свежий огурец", "огурцы в корзине", "фото огурца", "целый огурец", "cucumber", "green cucumber", "ripe cucumber", "cucumbers on plant", "fresh cucumber", "cucumbers in basket", "cucumber photo", "whole cucumber", "organic cucumber", "pickling cucumber", "english cucumber", "persian cucumber", "garden cucumber", "greenhouse cucumber", "органический огурец", "маринованный огурец", "английский огурец", "тепличный огурец", "огуречная плантация", "cucumber plantation", "огурцы в теплице"],
    "potato": ["картофель", "спелый картофель", "молодой картофель", "картошка в земле", "свежий картофель", "картошка в корзине", "фото картофеля", "целая картошка", "potato", "ripe potato", "young potato", "potatoes in ground", "fresh potato", "potatoes in basket", "potato photo", "whole potato", "organic potato", "russet potato", "red potato", "yellow potato", "garden potato", "farm potato", "органический картофель", "красный картофель", "желтый картофель", "садовый картофель", "фермерский картофель", "картофельное поле", "potato field", "картошка на грядке"],
    "background": ["деревянный стол", "wooden table", "кухонный стол", "kitchen table", "обеденный стол", "dining table", "лицо человека", "human face", "портрет человека", "human portrait", "мяч", "ball", "воздушный шар", "balloon", "елочный шар", "christmas ball", "декоративный шар", "decorative ball", "камни", "stones", "круглые камни", "round stones", "мыльные пузыри", "soap bubbles", "ваза", "vase", "керамическая посуда", "ceramic dishes", "тарелка", "plate", "пустая тарелка", "empty plate", "кухонная утварь", "kitchen utensils", "декоративные элементы", "decorative elements", "предметы интерьера", "interior items", "игрушки", "toys", "круглые игрушки", "round toys", "настольные предметы", "desktop items"]
}


In [ ]:
scrapper = ImageScrapper()

with tqdm(QUERIES.items(), desc="Scrapping", dynamic_ncols=True) as pbar:
    for group, queries in pbar:
        pbar.set_description(f"Scrapping {group}")
        scrapper.scrap(output_dir=f"../../dataset/raw/scrapped", 
                       queries=queries, 
                       group=group, 
                       limit=300)
